In [1]:
import numpy as np
import sympy as smp
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.integrate import cumulative_trapezoid
from tabulate import tabulate
import matplotlib
import piecewise_regression
import json

In [2]:
def mean(arr):
    return np.mean(arr)

def error(x):
    return x.std() / np.sqrt(x.size)

def weighted_error(x, err):
    
    x = np.asarray(x)
    err = np.asarray(err)
    
    average = np.sum(x/(err**2))/np.sum(1/err**2)
    
    average_error = 1/np.sqrt(np.sum(1/err**2))
    
    return average, average_error

def decimal(x):
    return x.replace(',', '.').encode()

def sum_err(x,sx):
    return np.sum(x),np.sqrt(np.sum([np.sum(i**2) for i in sx])) 

def diff_err(x,sx):
    return x[0]-x[1],np.sqrt(np.sum([np.sum(i**2) for i in sx])) 

def times_err(x,sx):
    return np.prod(x), np.prod(x)*np.sqrt(np.sum([np.sum((i/j)**2) for i,j in zip(sx,x)]))

#def frac_err(x,sx):
#    return x[0]/x[1], x[0]/x[1]*np.sqrt(np.sum((sx/x)**2))

def frac_err(x,sx):
    return x[0]/x[1], x[0]/x[1]*np.sqrt(np.sum([np.sum((i/j)**2) for i,j in zip(sx,x)]))
    #return [np.sum((i/j)**2) for i,j in zip(sx,x)]

In [3]:
def linear(x,a,b):
    return a*x + b

In [4]:
matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

# **ULOHA 1**

In [46]:
V = np.array([1,2,3,4,5,6,7,8,9,10])
V_real = np.array([0.9415,1.9355,2.9550,3.8875,4.9135,5.9040,6.8270,7.8135,8.7805,9.8230])
V_err = np.full(10,0.001)

## GRAF

In [47]:
params, err = curve_fit(linear, V, V_real, sigma=V_err)
a1,b1 = params
sa1,sb1 = np.sqrt(np.diag(err))

print("a: {} +- {}".format(a1,sa1))
print("b: {} +- {}".format(b1,sb1))

a: 0.9815151515151113 +- 0.003113274949725015
b: -0.02023333333555888 +- 0.019317334898310473


In [48]:
x_range = np.linspace(0,10,50)

plt.figure(figsize = (8,5))


plt.scatter(V,V_real,color = 'k',marker='x',label = 'Naměřené hodnoty napětí')
plt.plot(x_range,linear(x_range,a1,b1), color = 'k', linestyle = 'dashed', label = "Kalibrační křivka: " + r"$U_{real} = (0.982 \pm 0.003) U - (0.02 \pm 0.02)$")

#plt.legend(loc='upper left', fontsize=11)

handles, labels = plt.gca().get_legend_handles_labels()
order = [1,0]
plt.legend([handles[idx] for idx in order],[labels[idx] for idx in order], loc='upper left', fontsize=10)


#popisky os
plt.ylabel("U$_{real}$ " + r"[V]", fontsize=12)
plt.xlabel("$U$ [V]", fontsize=12)

plt.savefig('voltmetr.pdf', bbox_inches = 'tight')
plt.show()

C:\Users\stepa\AppData\Local\Temp/ipykernel_5760/2426513806.py:21: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


## Tabulka

In [49]:
T = np.array([V, V_real.round(3)]).T

print(tabulate(T, tablefmt="latex_booktabs"))

\begin{tabular}{rr}
\toprule
  1 & 0.942 \\
  2 & 1.936 \\
  3 & 2.955 \\
  4 & 3.888 \\
  5 & 4.914 \\
  6 & 5.904 \\
  7 & 6.827 \\
  8 & 7.814 \\
  9 & 8.78  \\
 10 & 9.823 \\
\bottomrule
\end{tabular}


# **ULOHA 2**

In [50]:
I = np.array([0.35,0.46,0.57,0.69,0.75,0.82,0.88,0.94,0.28,0.62])
I_real = np.array([0.35385,0.46740,0.57620,0.69010,0.74340,0.80475,0.85790,0.91940,0.28490,0.62205])
I_err = np.full(10,0.0001)

## GRAF

In [51]:
params, err = curve_fit(linear, I, I_real, sigma=I_err)
a2,b2 = params
sa2,sb2 = np.sqrt(np.diag(err))

print("a: {} +- {}".format(a2,sa2))
print("b: {} +- {}".format(b2,sb2))

a: 0.9555946077835115 +- 0.008948581287552673
b: 0.024236829393291304 +- 0.005996751714138521


In [52]:
def amper(x):
    return linear(x,a2,b2)

In [53]:
x_range = np.linspace(0,1,50)

plt.figure(figsize = (8,5))


plt.scatter(I,I_real,color = 'k',marker='x',label = 'Naměřené hodnoty proudu')
plt.plot(x_range,linear(x_range,a2,b2), color = 'k', linestyle = 'dashed', label = "Kalibrační křivka: " + r"$I_{real} = (0.955 \pm 0.009) I - (0.02 \pm 0.006)$")

#plt.legend(loc='upper left', fontsize=11)

handles, labels = plt.gca().get_legend_handles_labels()
order = [1,0]
plt.legend([handles[idx] for idx in order],[labels[idx] for idx in order], loc='upper left', fontsize=10)


#popisky os
plt.ylabel("I$_{real}$ " + r"[A]", fontsize=12)
plt.xlabel("$I$ [A]", fontsize=12)

plt.savefig('ampermetr.pdf', bbox_inches = 'tight')
plt.show()

C:\Users\stepa\AppData\Local\Temp/ipykernel_5760/3550735614.py:21: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


## Tabulka

In [54]:
T = np.array([I, I_real.round(4)]).T

print(tabulate(T, tablefmt="latex_booktabs"))

\begin{tabular}{rr}
\toprule
 0.35 & 0.3538 \\
 0.46 & 0.4674 \\
 0.57 & 0.5762 \\
 0.69 & 0.6901 \\
 0.75 & 0.7434 \\
 0.82 & 0.8048 \\
 0.88 & 0.8579 \\
 0.94 & 0.9194 \\
 0.28 & 0.2849 \\
 0.62 & 0.622  \\
\bottomrule
\end{tabular}


# ULOHA 3

In [55]:
R_dek = np.arange(100,1100,100)
U_x = np.array([0.0517,0.10145,0.15355,0.2039,0.25285,0.3042,0.35205,0.40625,0.45745,0.50675])
sU_x = np.full(10,0.001)
U_n = np.array([0.0550,0.0511,0.05165,0.05150,0.05115,0.5024,0.50195,0.5074,0.5078,0.50580])
sU_n = np.full(10,0.001)
R_n = np.array([100,100,100,100,100,1000,1000,1000,1000,1000])

In [56]:
R = U_x/U_n * R_n
sR = R*np.sqrt(sU_x**2/U_x**2 + sU_n**2/U_n**2)
R,sR

(array([  94.        ,  198.53228963,  297.28944821,  395.9223301 ,
         494.33040078,  605.49363057,  701.36467776,  800.65037446,
         900.84679007, 1001.87821273]),
 array([2.49535105, 4.35019583, 6.07274893, 7.9292396 , 9.86009145,
        2.3268836 , 2.43338712, 2.52469695, 2.65051009, 2.79862057]))

## GRAF

In [57]:
params, err = curve_fit(linear, R_dek, R, sigma=sR)
a3,b3 = params
sa3,sb3 = np.sqrt(np.diag(err))

print("a: {} +- {}".format(a3,sa3))
print("b: {} +- {}".format(b3,sb3))

a: 1.0077693949467599 +- 0.0031711428967027876
b: -4.4228069197834925 +- 2.1997213323754004


In [58]:
def ohm(x):
    return linear(x,a3,b3)

In [59]:
x_range = np.linspace(0,1000,50)

plt.figure(figsize = (8,5))


plt.scatter(R_dek,R,color = 'k',marker='x',label = 'Naměřené hodnoty odporu')
plt.plot(x_range,linear(x_range,a3,b3), color = 'k', linestyle = 'dashed', label = "Kalibrační křivka: " + r"$R_{real} = (1.007 \pm 0.003) R - (4 \pm 2)$")

#plt.legend(loc='upper left', fontsize=11)

handles, labels = plt.gca().get_legend_handles_labels()
order = [1,0]
plt.legend([handles[idx] for idx in order],[labels[idx] for idx in order], loc='upper left', fontsize=10)


#popisky os
plt.ylabel("R$_{real}$ " + "[$\Omega$]", fontsize=12)
plt.xlabel("$R$ [$\Omega$]", fontsize=12)

plt.savefig('dekada.pdf', bbox_inches = 'tight')
plt.show()

C:\Users\stepa\AppData\Local\Temp/ipykernel_5760/339597867.py:21: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


## Tabulka

In [60]:
T = np.array([R_dek, R, U_n.round(3),U_x.round(3), R_n]).T

print(tabulate(T, tablefmt="latex_booktabs"))

\begin{tabular}{rrrrr}
\toprule
  100 &   94     & 0.055 & 0.052 &  100 \\
  200 &  198.532 & 0.051 & 0.101 &  100 \\
  300 &  297.289 & 0.052 & 0.154 &  100 \\
  400 &  395.922 & 0.052 & 0.204 &  100 \\
  500 &  494.33  & 0.051 & 0.253 &  100 \\
  600 &  605.494 & 0.502 & 0.304 & 1000 \\
  700 &  701.365 & 0.502 & 0.352 & 1000 \\
  800 &  800.65  & 0.507 & 0.406 & 1000 \\
  900 &  900.847 & 0.508 & 0.457 & 1000 \\
 1000 & 1001.88  & 0.506 & 0.507 & 1000 \\
\bottomrule
\end{tabular}


# Uloha 4

In [61]:
I = amper(np.array([0.72,0.8,0.7,0.9,0.8,1,0.84,0.8,0.7,0.9]))
I_r = amper(np.array([0.72,0.8,0.7,0.9,0.8,1,0.84,0.8,0.7,0.9])/2)
R = ohm(np.array([112,114,113,117,113,119,115,113,112,117]))

In [62]:
T = np.array([I.round(2),I_r.round(2),R.round(0)]).T

print(tabulate(T, tablefmt="latex_booktabs"))

\begin{tabular}{rrr}
\toprule
 0.71 & 0.37 & 108 \\
 0.79 & 0.41 & 110 \\
 0.69 & 0.36 & 109 \\
 0.88 & 0.45 & 113 \\
 0.79 & 0.41 & 109 \\
 0.98 & 0.5  & 116 \\
 0.83 & 0.43 & 111 \\
 0.79 & 0.41 & 109 \\
 0.69 & 0.36 & 108 \\
 0.88 & 0.45 & 113 \\
\bottomrule
\end{tabular}


In [63]:
R0 = mean(R)
sR0 = error(R)
R0, sR0

(110.96678880162051, 0.7301982987726432)

# Uloha 5

In [64]:
U = volt(np.array([8,7,9,10,8,7,9,10,8.8,7.2]))
U_r = volt(np.array([8,7,9,10,8,7,9,10,8.8,7.2])/2)
R = ohm(np.array([3900,3800,3800,3800,3910,3820,3800,3800,3830,3840]))

NameError: name 'volt' is not defined

In [65]:
T = np.array([U.round(2),U_r.round(2),R.round(0)]).T

print(tabulate(T, tablefmt="latex_booktabs"))

\begin{tabular}{rrr}
\toprule
 7.67 & 3.85 & 108 \\
 6.71 & 3.37 & 110 \\
 8.62 & 4.32 & 109 \\
 9.58 & 4.8  & 113 \\
 7.67 & 3.85 & 109 \\
 6.71 & 3.37 & 116 \\
 8.62 & 4.32 & 111 \\
 9.58 & 4.8  & 109 \\
 8.43 & 4.23 & 108 \\
 6.9  & 3.46 & 113 \\
\bottomrule
\end{tabular}


In [66]:
R0 = mean(R)
sR0 = error(R)
R0, sR0

(110.96678880162051, 0.7301982987726432)